In [54]:
import torch
import torch.nn as nn
import torchvision

In [113]:
from model_partition import get_first_stage_resnet, get_second_stage_resnet, get_third_stage_resnet
model = torchvision.models.densenet169()

In [114]:
densenet = list(model.children())[0]
linear =  list(model.children())[1]

print("DenseBlock 1: ", len(list(densenet.denseblock1)))
print("DenseBlock 2: ", len(list(densenet.denseblock2)))
print("DenseBlock 3: ", len(list(densenet.denseblock3)))
print("DenseBlock 4: ", len(list(densenet.denseblock4)))

DenseBlock 1:  6
DenseBlock 2:  12
DenseBlock 3:  32
DenseBlock 4:  32


In [ ]:
densenet = torchvision.models.densenet169()
dnet = a.features
dnet[4]

In [108]:
densenet_model = torchvision.models.densenet169()

densenet = densenet_model.features
classifier = densenet_model.classifier

# densenet.transition1

model1 = nn.Sequential(
    densenet.conv0,
    densenet.norm0,

    densenet.relu0,
    densenet.pool0,

    densenet.denseblock1,
    densenet.transition1,

    densenet.denseblock2,
    densenet.transition2,

    *list(densenet.denseblock3.children())[0:8]
)


model2 = nn.Sequential(
    *list(densenet.denseblock3.children())[8:],
    densenet.transition3,
    *list(densenet.denseblock4.children())[0:4]
)

in_features_last_layer = classifier.in_features
out_features_last_layer = 10

model3 = nn.Sequential(
    *list(densenet.denseblock4.children())[4:],
    densenet.norm5,
    nn.Linear(in_features = in_features_last_layer, out_features = out_features_last_layer)
)
# print(model1)
print(model3)




Sequential(
  (0): _DenseLayer(
    (norm1): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv1): Conv2d(768, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (1): _DenseLayer(
    (norm1): BatchNorm2d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv1): Conv2d(800, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (2): _DenseLayer(
    (norm1): BatchNorm2d(832, eps=1e-05, momentum=0.1, affine=True, track_runn

In [ ]:
from torchvision.models.densenet import _DenseBlock
import torchvision
import torch.nn as nn
densenet_model = torchvision.models.densenet169()

model = densenet_model.features
classifier = densenet_model.classifier

from onnx_file import create_onnx_from_model, create_engine
import torch


In [3]:
# Stage 0 - denseblock 1 + denseblock2 = denseblock 3[1: 9]
# Layer 3 - 9 layers. starting at layer 1. num_input_features = 256 
stage0 = nn.Sequential()

stage0_layer3_1 = _DenseBlock(num_layers=9, num_input_features=256, growth_rate=32, drop_rate=0, bn_size=4)

# Add for stage 0
stage0.add_module("conv0", model.conv0)
stage0.add_module("norm0", model.norm0)
stage0.add_module("relu0", model.relu0)
stage0.add_module("pool0", model.pool0)

stage0.add_module("denseblock1", model.denseblock1)
stage0.add_module("transition1", model.transition1)

stage0.add_module("denseblock2", model.denseblock2)
stage0.add_module("trasition2", model.transition2)

stage0.add_module("custom_denseblock3_stage0", stage0_layer3_1)

stage0_input = torch.randn(16, 3, 32, 32)
engine_input = tuple(stage0_input.shape)

create_onnx_from_model(stage0, "sample_stage0.onnx", input_tensor=stage0_input)
# create_engine("", "sample_stage0_tensort.trt", engine_input, engine_input, engine_input, batch_size=16 )

# print(stage0) 

[ONNX] Exporting ONNX model sample_stage0.onnx
[ONNX] Exported to: sample_stage0.onnx


In [14]:
# stage 1 - denseblock3[10:32] + denseblock[1:5]
# denseblock3 starts at layer 5 - num_input_features = 544 (32 - 9 = 23 layers)
# denseblock4 starts at layer 1 - num_input_features = 640 ( 5 layers)

stage1 = nn.Sequential()

stage1_layer3 = _DenseBlock(num_layers=23, num_input_features=544, bn_size=4, growth_rate=32, drop_rate=0)
stage1_transtition3 = model.transition3 
stage1_layer4 = _DenseBlock(num_layers=5, num_input_features=640, bn_size=4, growth_rate=32, drop_rate=0)

stage1.add_module("customm_denseblock3_stage1", stage1_layer3)
stage1.add_module("transiton3", stage1_transtition3)
stage1.add_module("custom_denseblock4_stage1", stage1_layer4)


stage0_input = torch.randn(16, 3, 32, 32)
stage1_input = stage0(stage0_input)

create_onnx_from_model(stage1, "sample_stage1.onnx", input_tensor=stage1_input)

# print(stage1)


[ONNX] Exporting ONNX model sample_stage1.onnx
[ONNX] Exported to: sample_stage1.onnx


In [5]:
# Stage2 - denseblock4[6:32] (startsat layer 6 - num_input_features = 800, num_layers = 27)
import torch.functional as F
stage2= nn.Sequential()


stage2_layer4 = _DenseBlock(num_layers = 27, num_input_features=800, bn_size=4, growth_rate=32, drop_rate=0)

stage2.add_module("custom_denseblock_stage2", stage2_layer4)
stage2.add_module("norm5", model.norm5)
stage2.add_module("custm_relu", nn.ReLU(inplace=True) )
stage2.add_module("custm_adp_avg_pool_2d", nn.AdaptiveAvgPool2d((1, 1)))
stage2.add_module("custm_flatten", nn.Flatten(1))
stage2.add_module("classifier", classifier)


stage0_input = torch.randn(16, 3, 32, 32)
stage1_input = stage0(stage0_input)
stage2_input = stage1(stage1_input)

stage2_output = stage2(stage2_input)
print(stage2_output.shape)

create_onnx_from_model(stage2, "sample_stage_2.onnx", input_tensor=stage2_input)

# print(stage2)

torch.Size([16, 1000])
[ONNX] Exporting ONNX model sample_stage_2.onnx
[ONNX] Exported to: sample_stage_2.onnx


In [15]:
create_engine(f"sample_stage1.onnx", "engine_sample.trt", (16,3,32,32), (16,3,32,32), (16,3,32,32))

[ONNX_parse] ONNX file sample_stage1.onnx parsed successfully
[TRT_ENG] Building engine for: sample_stage1.onnx


RuntimeError: Engine build failed